In [2]:
import xml.etree.ElementTree as ET

In [3]:
# Parse the XML file
tree = ET.parse("cwe.xml")
root = tree.getroot()

# Define namespace
ns = {"cwe": "http://cwe.mitre.org/cwe-7"}

cwe_dict = {weakness.get("ID"): weakness for weakness in root.findall(".//cwe:Weakness", ns)}

In [8]:
# Function that tells type
def get_type(cwe_id:str):
    weakness = cwe_dict.get(cwe_id)
    abstraction = weakness.get("Abstraction")
    return abstraction

# Function that find parent based on specified id
def find_parents(cwe_id:str):    
    result = {"Pillar": [], "Class": [], "Base": [], "Variant": []}
    
    current_cwe_id = cwe_id
    while current_cwe_id:
        # Find the Weakness element
        weakness = cwe_dict.get(current_cwe_id)
        if not weakness:
            return None  # If CWE is not found, stop the search

        abstraction = weakness.get("Abstraction")
        if ((result.get(abstraction, None)) is not None):
            result[abstraction].append(current_cwe_id)
        
        # Check if the weakness abstraction is "Pillar"
        if abstraction == "Pillar":
            break  # Stop search if it's a Pillar

        # Find the Parent CWE (ChildOf relation)
        related_weaknesses = weakness.find("cwe:Related_Weaknesses", ns)
        if related_weaknesses is not None:
            for related in related_weaknesses.findall("cwe:Related_Weakness", ns):
                if related.get("Nature") == "ChildOf":
                    current_cwe_id = related.get("CWE_ID")
    
    return result

tmp = find_parents("82")

In [18]:
import os
import pickle



{'file_change_id': '15303494186538', 'true_vuln': True, 'actual_cwe': 'CWE-918', 'result': {'is_this_vuln': True, 'vuln_code_part': 'const [ip, port, family] = ActiveConnector.prototype.setupConnection.args[0];', 'reason': "The code is vulnerable to CWE-79: Improper Neutralization of Input During Web Page Generation ('Cross-site Scripting'). The `setupConnection` method is being called with user-controlled input (the IP address and port number) without proper sanitization or validation. This allows an attacker to inject malicious data, potentially leading to a cross-site scripting attack.", 'cwe': {'cwe_id': 'CWE-79', 'name': "Improper Neutralization of Input During Web Page Generation ('Cross-site Scripting')", 'description': 'The software does not neutralize or incorrectly neutralizes user-controllable input to HTML output. As a result, an attacker can inject malicious scripts into the web page.'}}, 'model': 'llama3.1:8b', 'version': 1, 'error': False, 'time': 4.8003373200008355}
